In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [24]:
# !pip install graphviz

In [2]:
#读取原始数据
Obesity = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")
Obesity.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
#将双字符串变量转化为0-1变量，将因变量转化为1-7
Obesity.loc[Obesity['Gender'] == 'Female','Gender'] = 0
Obesity.loc[Obesity['Gender'] == 'Male','Gender'] = 1
Obesity.loc[Obesity['family_history_with_overweight'] == 'yes','family_history_with_overweight'] = 1
Obesity.loc[Obesity['family_history_with_overweight'] == 'no','family_history_with_overweight'] = 0
Obesity.loc[Obesity['FAVC'] == 'yes','FAVC'] = 1
Obesity.loc[Obesity['FAVC'] == 'no','FAVC'] = 0
Obesity.loc[Obesity['SMOKE'] == 'yes','SMOKE'] = 1
Obesity.loc[Obesity['SMOKE'] == 'no','SMOKE'] = 0
Obesity.loc[Obesity['SCC'] == 'yes','SCC'] = 1
Obesity.loc[Obesity['SCC'] == 'no','SCC'] = 0
Obesity.loc[Obesity['NObeyesdad'] == 'Insufficient_Weight','NObeyesdad'] = 1
Obesity.loc[Obesity['NObeyesdad'] == 'Normal_Weight','NObeyesdad'] = 2
Obesity.loc[Obesity['NObeyesdad'] == 'Overweight_Level_I','NObeyesdad'] = 3
Obesity.loc[Obesity['NObeyesdad'] == 'Overweight_Level_II','NObeyesdad'] = 4
Obesity.loc[Obesity['NObeyesdad'] == 'Obesity_Type_I','NObeyesdad'] = 5
Obesity.loc[Obesity['NObeyesdad'] == 'Obesity_Type_II','NObeyesdad'] = 6
Obesity.loc[Obesity['NObeyesdad'] == 'Obesity_Type_III','NObeyesdad'] = 7
Obesity.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,0,21.0,1.62,64.0,1,0,2.0,3.0,Sometimes,0,2.0,0,0.0,1.0,no,Public_Transportation,2
1,0,21.0,1.52,56.0,1,0,3.0,3.0,Sometimes,1,3.0,1,3.0,0.0,Sometimes,Public_Transportation,2
2,1,23.0,1.80,77.0,1,0,2.0,3.0,Sometimes,0,2.0,0,2.0,1.0,Frequently,Public_Transportation,2
3,1,27.0,1.80,87.0,0,0,3.0,3.0,Sometimes,0,2.0,0,2.0,0.0,Frequently,Walking,3
4,1,22.0,1.78,89.8,0,0,2.0,1.0,Sometimes,0,2.0,0,0.0,0.0,Sometimes,Public_Transportation,4


In [4]:
#将多字符串变量转化为虚拟变量
Obesity_CAEC = pd.get_dummies(Obesity['CAEC'])
Obesity_CAEC.columns = ['CAEC_Always','CAEC_Frequently','CAEC_Sometimes','CAEC_no']
Obesity_CALC = pd.get_dummies(Obesity['CALC'])
Obesity_CALC.columns = ['CALC_Always','CALC_Frequently','CALC_Sometimes','CALC_no']
Obesity_MTRANS = pd.get_dummies(Obesity['MTRANS'])
Obesity_MTRANS.columns = ['MTRANS_Automobile','MTRANS_Bike','MTRANS_Motorbike','MTRANS_Public_Transportation','MTRANS_Walking']

In [5]:
#将转化后的变量重新组合
Obesity1 = Obesity.drop(columns=['CAEC','CALC','MTRANS','NObeyesdad'])
Obesity1 = pd.concat([Obesity1,Obesity_CAEC,Obesity_CALC,Obesity_MTRANS,Obesity['NObeyesdad']],axis=1)
Obesity1.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,SMOKE,CH2O,...,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking,NObeyesdad
0,0,21.0,1.62,64.0,1,0,2.0,3.0,0,2.0,...,0,0,0,1,0,0,0,1,0,2
1,0,21.0,1.52,56.0,1,0,3.0,3.0,1,3.0,...,0,0,1,0,0,0,0,1,0,2
2,1,23.0,1.80,77.0,1,0,2.0,3.0,0,2.0,...,0,1,0,0,0,0,0,1,0,2
3,1,27.0,1.80,87.0,0,0,3.0,3.0,0,2.0,...,0,1,0,0,0,0,0,0,1,3
4,1,22.0,1.78,89.8,0,0,2.0,1.0,0,2.0,...,0,0,1,0,0,0,0,1,0,4


In [36]:
Obesity1.columns

Index(['Gender', 'Age', 'Height', 'Weight', 'family_history_with_overweight',
       'FAVC', 'FCVC', 'NCP', 'SMOKE', 'CH2O', 'SCC', 'FAF', 'TUE',
       'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no',
       'CALC_Always', 'CALC_Frequently', 'CALC_Sometimes', 'CALC_no',
       'MTRANS_Automobile', 'MTRANS_Bike', 'MTRANS_Motorbike',
       'MTRANS_Public_Transportation', 'MTRANS_Walking', 'NObeyesdad'],
      dtype='object')

In [70]:
x=input('please input a type:')
y=input('please input a condition:')
z=a[x]+y
if type(x)==str:
    if x in Obesity.columns:
        a=pd.DataFrame(Obesity[x],columns=[x])
        a['NObeyesdad']=Obesity['NObeyesdad']
        print('体重正常的样本数为：%d'%a.loc[a['NObeyesdad']==2&&z].iloc[:,0].size)
        print('总样本数为：%d'%a.iloc[:,0].size)
    else:print('请输入正确的特征名')
else:print('请输入字符串')

SyntaxError: invalid syntax (<ipython-input-70-857cd9f209cd>, line 8)

In [6]:
#将重新组合后的df转化为array
Obesity_data = np.array(Obesity1)
print(Obesity_data.shape)

(2111, 27)


In [7]:
#选取X,y
X = Obesity_data[:,0:-1]
y = Obesity_data[:,-1].astype('int')

In [8]:
#X,y的shape
print(X.shape)
print(y.shape)

(2111, 26)
(2111,)


In [9]:
#将X,y分为训练集和测试集
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

In [10]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

In [11]:
#建立tree_model的最大深度为5
tree_Obesity = DecisionTreeClassifier(max_depth=5)

In [12]:
#用训练集训练模型
tree_Obesity.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [25]:
#输出训练好的模型
import graphviz
dot_data = export_graphviz(tree_Obesity,feature_names= ['Gender','Age','Height','Weight','family_history_with_overweight','FAVC','FCVC','NCP','CAEC_Always','CAEC_Frequently','CAEC_Sometimes','CAEC_no','SMOKE','CH2O','SCC','FAF','TUE','CALC_Always','CALC_Frequently','CALC_Sometimes','CALC_no','MTRANS_Automobile','MTRANS_Bike','MTRANS_Motorbike','MTRANS_Public_Transportation','MTRANS_Walking'], class_names = ['1','2','3','4','5','6','7']) 
tree_graph = graphviz.Source(dot_data) 
tree_graph.render("Obesity") 

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'Obesity'], make sure the Graphviz executables are on your systems' PATH

In [14]:
#用测试集输出混淆矩阵和整体准确度
y_predict = tree_Obesity.predict(X_test)
CM_tree = confusion_matrix(y_test, y_predict)
print(CM_tree)
print(accuracy_score(y_test, y_predict))

[[50 10  0  0  0  0  0]
 [ 6 58 18  7  0  0  0]
 [ 0  1 34 36  0  0  0]
 [ 0  0  1 54 24  0  0]
 [ 0  0  1  0 77  1  0]
 [ 0  0  0  0  3 68  0]
 [ 0  0  0  0  0  0 79]]
0.7954545454545454


In [15]:
#建立惩罚矩阵
penalty = np.array([[0,1,2,3,4,5,6],[1,0,1,2,3,4,5],[2,1,0,1,2,3,4],[3,2,1,0,1,2,3],[4,3,2,1,0,1,2],[5,4,3,2,1,0,1],[6,5,4,3,2,1,0,]])
print(penalty)

[[0 1 2 3 4 5 6]
 [1 0 1 2 3 4 5]
 [2 1 0 1 2 3 4]
 [3 2 1 0 1 2 3]
 [4 3 2 1 0 1 2]
 [5 4 3 2 1 0 1]
 [6 5 4 3 2 1 0]]


In [16]:
#tree_model的惩罚误差
np.sum(np.multiply(CM_tree,penalty))/np.sum(CM_tree)

0.2196969696969697

In [17]:
#为树的最大深度进行交叉验证的参数调优
from sklearn.model_selection import GridSearchCV
param_grid = {'max_depth':range(1, 7)}
grid = GridSearchCV(DecisionTreeClassifier(random_state=0),param_grid=param_grid,cv=10)
grid.fit(X_train, y_train)
print(grid.best_params_)
print('Best Overal Accuracy: %.2f' %grid.best_score_)

{'max_depth': 6}
Best Overal Accuracy: 0.88


In [18]:
#建立best_tree_model的最大深度为6
tree_Obesity_best = DecisionTreeClassifier(max_depth=6)

In [19]:
#用训练集训练模型
tree_Obesity_best.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=6, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [26]:
#输出训练好的模型
dot_data_best = export_graphviz(tree_Obesity_best,feature_names= ['Gender','Age','Height','Weight','family_history_with_overweight','FAVC','FCVC','NCP','CAEC_Always','CAEC_Frequently','CAEC_Sometimes','CAEC_no','SMOKE','CH2O','SCC','FAF','TUE','CALC_Always','CALC_Frequently','CALC_Sometimes','CALC_no','MTRANS_Automobile','MTRANS_Bike','MTRANS_Motorbike','MTRANS_Public_Transportation','MTRANS_Walking'], class_names = ['1','2','3','4','5','6','7']) 
tree_graph_best = graphviz.Source(dot_data_best) 
tree_graph_best.render("Obesity_best") 

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'Obesity_best'], make sure the Graphviz executables are on your systems' PATH

In [21]:
#用测试集输出混淆矩阵和整体准确度
y_predict_best = tree_Obesity_best.predict(X_test)
CM_tree_best = confusion_matrix(y_test, y_predict_best)
print(CM_tree_best)
print(accuracy_score(y_test, y_predict_best))

[[58  2  0  0  0  0  0]
 [13 53 23  0  0  0  0]
 [ 0  6 57  8  0  0  0]
 [ 0  3  7 67  2  0  0]
 [ 0  0  1  1 76  1  0]
 [ 0  0  0  0  1 70  0]
 [ 0  0  0  0  0  0 79]]
0.8712121212121212


In [22]:
#tree_model的惩罚误差
np.sum(np.multiply(CM_tree_best,penalty))/np.sum(CM_tree_best)

0.13636363636363635

In [23]:
#随机森林模型
from sklearn.ensemble import RandomForestClassifier

train_scores = []
test_scores = []

rf = RandomForestClassifier()
estimator_range = range(1, 100, 5)

for n_estimators in estimator_range:
    rf.n_estimators = n_estimators
    rf.fit(X_train, y_train)
    train_scores.append(rf.score(X_train, y_train))
    test_scores.append(rf.score(X_test, y_test))
    
print(train_scores)
print(test_scores)

[0.9216677195198989, 0.9917877447883765, 0.9993682880606444, 0.9993682880606444, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.7727272727272727, 0.9128787878787878, 0.9185606060606061, 0.9185606060606061, 0.928030303030303, 0.9356060606060606, 0.9261363636363636, 0.9242424242424242, 0.9318181818181818, 0.9318181818181818, 0.9337121212121212, 0.928030303030303, 0.9337121212121212, 0.9393939393939394, 0.9337121212121212, 0.9393939393939394, 0.946969696969697, 0.9375, 0.9450757575757576, 0.9488636363636364]
